# Configuration

In [ ]:
#%matplotlib notebook
%matplotlib inline

import sklearn.cluster as skcl;
import pandas as pd;
import matplotlib.pyplot as plt
import utils.feature_normalization as fn
from bisect import bisect
import seaborn as sns
import numpy as np


## Read Data

In [ ]:
features = pd.read_csv( filepath_or_buffer="features.txt" , sep="\t" )
features = features.query( 'longest_stop_s >= 0' )
print( features.head( 3 ) );

## Plot data

In [ ]:
feature_names = features.columns.values

# start at 1 because 0 is agent id
#for i in range(1, len( feature_names ) ):
#    #print( "index i=" , i )
#    for j in range( i + 1 , len( feature_names ) ):
#        #print( "index j=" , j )
#        feature1 = feature_names[ i ]
#        feature2 = feature_names[ j ]
#        plt.scatter( features[ feature1 ] , features[ feature2 ] )
#        plt.xlabel( feature1 )
#        plt.ylabel( feature2 )
#        plt.show()
        
grid = sns.PairGrid( features , vars=feature_names[1:] )
grid.map_diag( plt.hist )
grid.map_offdiag( plt.scatter )


## Normalize features


In [ ]:
norm_features = fn.normalize_features( features )

print( norm_features.head() )


## Train clustering algorithm

### K Means

In [ ]:
kmeans_per_size = {}

for n_clusters in range(2,11):
    kmeans = skcl.KMeans(n_clusters=n_clusters)
    kmeans.fit(norm_features)
    kmeans_per_size[ n_clusters ] = kmeans

n_cluster_list = list( kmeans_per_size.keys() )
score = list( map( lambda k: k.score( norm_features ), kmeans_per_size.values() ) )
plt.plot( n_cluster_list , score )

In [ ]:
# from looking at score, 5 seems to be a good choice ("elbow")
pred = kmeans_per_size[ 5 ].predict( norm_features )
# start at 1 because 0 is agent id
#for i in range(1, len( feature_names ) ):
#    #print( "index i=" , i )
#    for j in range( i + 1 , len( feature_names ) ):
#        #print( "index j=" , j )
#        feature1 = feature_names[ i ]
#        feature2 = feature_names[ j ]
#        plt.scatter( features[ feature1 ] , features[ feature2 ] , c=pred , alpha=.5 )
#        plt.xlabel( feature1 )
#        plt.ylabel( feature2 )
#        plt.show()

for i in range(5):
    print( "cluster " , i )
    f = features[ pred == i ]
    print( "number of elements: " , len( f.index ) )
    print( f.describe().transpose() )
    grid = sns.PairGrid( f , vars=feature_names[1:] )
    grid.map_diag( plt.hist )
    grid.map_upper( plt.scatter )
    #grid.map_lower( sns.kdeplot , cmap="Blues_d" )


## Meaningful clustering

Clustering based on meaningful boundaries.

First define some charging times. From https://www.clippercreek.com/wp-content/uploads/2016/04/TIME-TO-CHARGE-20170706_FINAL-LOW-RES.jpg, full charging times range from 2 to 70 hours depending on vehicle and charging station.

In [ ]:
charge_time_thresholds = np.array( [ 3600 * 2 ** i for i in range(1,10) if (2 ** i <= 24) ] )
print( charge_time_thresholds )

Then define range.
- Range of nissan leaf (most common EV): 135km https://en.wikipedia.org/wiki/Nissan_Leaf
- Tesla 85D: 270 miles = 434km https://www.tesla.com/fr_CH/blog/driving-range-model-s-family?redirect=no

Range depends on lots of factors, so we just use a few thresholds starting at 50km up to 400km

In [ ]:
range_thresholds = np.array( [ 50 * 1000 * 2 ** i for i in range(4) ] )
print( range_thresholds / 1000 )

Now, just generate one label per combination and compute labels

In [ ]:
def find_threshold( t , thresholds , unit ):
    if t == len( thresholds ):
        return ''.join( ( "> " , str( thresholds[ t - 1 ] ) , unit ) )
    if t == 0:
        return ''.join( ( "[0 , " , str( thresholds[ t ] ) , unit , "]" ) )
    return ''.join( ( "[" , str( thresholds[ t - 1 ] ) , " , " , str( thresholds[ t ] ) , unit , "]" ) )

def comp_label( range_classes , charge_time_classes ):
    m = map( lambda r , t: "".join(
                ( "range_" , find_threshold( r ,  range_thresholds / 1000 , "km" ) ,
                  "-time_" , find_threshold( t , charge_time_thresholds / 3600 , "h" ) ) ) ,
             range_classes , charge_time_classes ) 
    return np.array( list( m ) ) 

pred_meaning = (features.assign( range_class = list( map( lambda x: bisect( range_thresholds , x ) , features.longest_trip_m ) ),
                  charge_time_class = list( map( lambda x: bisect( charge_time_thresholds , x ) , features.longest_stop_9_16_s )))
        .assign( range_label = lambda x: list( map( lambda t: find_threshold( t , range_thresholds / 1000 , "km" ) , x.range_class ) ),
                 charge_time_label = lambda x: list( map( lambda t: find_threshold( t , charge_time_thresholds / 3600 , "h" ) , x.charge_time_class ) ) )
        .assign( label = lambda x: comp_label( x.range_class , x.charge_time_class)))


In [ ]:
# start at 1 because 0 is agent id
#for i in range(1, len( feature_names ) ):
#    #print( "index i=" , i )
#    for j in range( i + 1 , len( feature_names ) ):
#        #print( "index j=" , j )
#        feature1 = feature_names[ i ]
#        feature2 = feature_names[ j ]
#        plt.scatter( features[ feature1 ] , features[ feature2 ] , c=pred_meaning.label  )
#        plt.xlabel( feature1 )
#        plt.ylabel( feature2 )
#        plt.show()

grid = sns.PairGrid( features.assign( label = pred_meaning.label ) , vars=feature_names[1:] , hue='label' )
grid.map_diag( plt.hist )
grid.map_upper( plt.scatter , alpha=.5 )
grid.add_legend()

In [ ]:
crosstab_clusters = pd.crosstab( pred_meaning.range_label , pred_meaning.charge_time_label ) 

print( crosstab_clusters )
#crosstab_clusters.to_excel( 'crosstab_meaningful_clusters.xlsx' )
#crosstab_clusters.to_csv( 'crosstab_meaningful_clusters.csv' )